# Introduction

OilyGiant mining company is trying to find the best place for a new well. In order to find the best location, we will have to perform the following; collect the woil well parameters, build a model for predicting the volume of reserves in the new wells, pick the oil wells with the hghest estiamged calues, and pick the region with the highest total profile margin. By using the bootstrapping technique we will be able to analyze the potaitnal profit and rsiks for choosing the right oil region. 


# Import Libraries

In [81]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight
from sklearn import tree
from sklearn.metrics import f1_score, roc_auc_score, classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from xgboost import XGBClassifier

# Load and Preprocess Data Sets

In [82]:
#Load Data

data0 = pd.read_csv('/datasets/geo_data_0.csv')

data1 = pd.read_csv('/datasets/geo_data_1.csv')

data2 = pd.read_csv('/datasets/geo_data_2.csv')

datasets = [data0, data1, data2]

In [83]:
#View the data in full table

display(data0.head())

display(data1.head())

display(data2.head())

#The number of rows and columns

data0.shape

data1.shape

data2.shape

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


(100000, 5)

In [84]:
#Checking columns in dataframe

print(data0.columns)

print(data1.columns)

print(data2.columns)

Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')


In [85]:
#Checking for duplicates

duplicates = data0.duplicated()

print(duplicates.sum())

duplicates = data1.duplicated()

print(duplicates.sum())

duplicates = data2.duplicated()

print(duplicates.sum())

0
0
0


In [86]:
#Check for missing values

missing_values = data0.isnull().sum()

print(missing_values)

missing_values = data1.isnull().sum()

print(missing_values)

missing_values = data2.isnull().sum()

print(missing_values)

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [87]:
#Checking for data types

print(data0.dtypes)

print(data1.dtypes)

print(data2.dtypes)

id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object


# Train Models

In [88]:
#Function to Train Model and Get Predictions

def train_model(data):
    
    X = data.drop(columns=['id', 'product'])
    
    y = data['product']
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)
    
    model = LinearRegression()
    
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_val)
    
    return model, predictions, y_val

In [89]:
model0, predictions0, y_val0 = train_model(data0)

model1, predictions1, y_val1 = train_model(data1)

model2, predictions2, y_val2 = train_model(data2)

In [90]:
#Function to Calculate Average Volume of Predicted Reserves and RMSE

def calculate_metrics(predictions, y_val):
    
    #Calculate RMSE
    
    rmse = (y_val, predictions, squared=False)
    
    #Calculate average volume of predicted reserves.
    
    avg_volume = predictions.mean()
    
    return avg_volume, rmse

#Train and calculate metrics for each model

model0, predictions0, y_val0 = train_model(data0)

avg_volume0, rmse0 = calculate_metrics(predictions0, y_val0)

model1, predictions1, y_val1 = train_model(data1)

avg_volume1, rmse1 = calculate_metrics(predictions1, y_val1)

model2, predictions2, y_val2 = train_model(data2)

avg_volume2, rmse2 = calculate_metrics(predictions2, y_val2)

#Print the results

print(f"Model 0 - Average Predicted Reserve Volume: {avg_volume0:.2f}, RMSE: {rmse0:.2f}")

print(f"Model 1 - Average Predicted Reserve Volume: {avg_volume1:.2f}, RMSE: {rmse1:.2f}")

print(f"Model 2 - Average Predicted Reserve Volume: {avg_volume2:.2f}, RMSE: {rmse2:.2f}")

Model 0 - Average Predicted Reserve Volume: 92.40, RMSE: 37.76
Model 1 - Average Predicted Reserve Volume: 68.71, RMSE: 0.89
Model 2 - Average Predicted Reserve Volume: 94.77, RMSE: 40.15


# Trained Model Analysis

Model 1 is the most accurate, with a very low RMSE of 0.89, which means the predictions are very close to the actual values, with an average error of only 0.89 units. However, it predicts the lowest average reserve volume at 68.71 units, which means it might be underestimating the reserves compared to the other models. Model 0 has a higher RMSE of 37.76, indicating that its predictions are less accurate. Its average predicted reserve volume is 92.40, which is higher than Model 1's but lower than Model 2's. Model 2 has the highest predicted reserve volume at 94.77 units, but it also has the highest RMSE of 40.15, which means its predictions are the least accurate with an average error of 40.15 units. In conclusion, Model 1 is the most accurate, but it underestimates the reserve volume, while Model 2 overestimates the volume and is the least accurate. Model 0 falls somewhere in between in both prediction volume and accuracy.

# Calculate Profit

In [91]:
#$100 million for develeoping wells

budget = 100_000_000

#200 wells in each region

wells_to_drill = 200

#$4.5 per barrel

barrel_price = 4.5

#Product is in the thousands

total_barrel_revenue = 4500 

In [92]:
#How much each well needs to prouce to not lose money

barrels_needed = budget / (wells_to_drill * total_barrel_revenue)

print(f"Minimum barrels needed per well: {barrels_needed:.2f}")

Minimum barrels needed per well: 111.11


In [93]:
#Compare minimum barrel to average oil reserves per region

avg0 = data0['product'].mean()

avg1 = data1['product'].mean()

avg2 = data2['product'].mean()

print(f"Region 0 average reserves: {avg0:.2f}")

print(f"Region 1 average reserves: {avg1:.2f}")

print(f"Region 2 average reserves: {avg2:.2f}")

Region 0 average reserves: 92.50
Region 1 average reserves: 68.83
Region 2 average reserves: 95.00


In [94]:
def calculate_profit(predictions, actual_reserves, wells_to_drill=200, price_per_barrel=4.5, cost_per_well=500_000):
    predictions = pd.Series(predictions)
    
    #Select top 200 wells based on predictions
    
    top_well_indices = predictions.nlargest(wells_to_drill).index
    
    #Extract actual reserves for those wells
    
    total_oil_extracted = actual_reserves.iloc[top_well_indices].sum()
    
    #Calculate revenue
    
    revenue = total_oil_extracted * price_per_barrel * 1000
    
    total_cost = cost_per_well * wells_to_drill
    
    #Compute profit
    
    total_profit = revenue - total_cost
    
    return total_profit

# Calculate Profit Analysis:
The minium number of barrells needed for each well is at least 111.11. We used the mean of each data set to compare minimum barrel to average oil reserves per region:
Region 0 average reserves: 92.50
Region 1 average reserves: 68.83
Region 2 average reserves: 95.00

# Bootstrapping 

In [95]:
def bootstrap_profit(predictions, actual_reserves, price_per_barrel=4.5, budget=100_000_000, wells_to_drill=200, num_samples=500):
    np.random.seed(42) 
    
    profits = []
    
    for _ in range(num_samples): 
        
        #Bootstrap sampling
        
        sample_indices = np.random.choice(len(predictions), size=500, replace=True)
        
        #Select the predictions and actuals corresponding to the sample
        
        sample_predictions = predictions[sample_indices]
        
        #Use .iloc to ensure correct indexing
        
        sample_actuals = actual_reserves.iloc[sample_indices]  
        
        #Calculate profit using the calculate_profit function
        
        profit = calculate_profit(sample_predictions, sample_actuals)
        
        profits.append(profit)
    
    #Convert profits list into a NumPy array for analysis
    
    profits = np.array(profits)
    
    #Compute statistics
    
    avg_profit = profits.mean()
    
    lower_95, upper_95 = np.percentile(profits, [2.5, 97.5])
    
    #Risk of financial loss (percentage of times profit is negative)
    
    loss_risk = (profits < 0).mean() * 100  

    return avg_profit, (lower_95, upper_95), loss_risk

In [96]:
#Running bootstrapping for each region using validation data

avg_profit0, conf_interval0, loss_risk0 = bootstrap_profit(predictions0, y_val0)

avg_profit1, conf_interval1, loss_risk1 = bootstrap_profit(predictions1, y_val1)

avg_profit2, conf_interval2, loss_risk2 = bootstrap_profit(predictions2, y_val2)

# Analysis of Final Region

In [97]:
#Print results for each region

for i, (avg_profit, conf_interval, loss_risk) in enumerate([(avg_profit0, conf_interval0, loss_risk0),
                                                           (avg_profit1, conf_interval1, loss_risk1),
                                                           (avg_profit2, conf_interval2, loss_risk2)]):
    print(f"Region {i}:")
    
    print(f" Average Profit: ${avg_profit:.2f}")
    
    print(f" 95% Confidence Interval: ${conf_interval[0]:,.2f} to ${conf_interval[1]:,.2f}")
    
    print(f" Risk of Loss: {loss_risk:.2f}%\n")

Region 0:
 Average Profit: $4027065.30
 95% Confidence Interval: $-962,478.94 to $8,898,624.95
 Risk of Loss: 4.80%

Region 1:
 Average Profit: $4487290.37
 95% Confidence Interval: $458,001.97 to $8,324,694.77
 Risk of Loss: 2.20%

Region 2:
 Average Profit: $3801587.77
 95% Confidence Interval: $-1,286,135.66 to $8,600,086.00
 Risk of Loss: 6.20%



# Conclusion
Based on the results, Region 1 seems like the best option because it has the highest average profit of 4.49 million and the lowest risk of lost of 2.20 percent. Region 0 is may be a good choice, but its average profit was lower calculating at 4.03 million and the risk of loss was higher of 4.80 percent. Region 2 appeared to be the rickiest, with the lowest average profit of $3.8 million and the highest risk of loss of 6.20 percent, making it the worst option for Oily. 

Overall, Region 1 would be the safest investment for stakeholders and investers giving the highest return with the lowest risk. The region to avoid would be Region 2. 